Задача поиск похожих по эмбеддингам**
Скачиваем датасет 

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

что надо сделать 

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [1]:
pip install wget


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText

from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm

In [6]:
RANDOM_STATE = 42
NUM_THREADS = 12

Загрузка и подготовка данных

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/NLP/3 урок/gazeta.csv', usecols=['text', 'title'])
df.head()

,title,text
0,Госдума сокращает срок действия ставки экспорт...,Госдума приняла сегодня в первом чтении и сраз...
1,Японские физики повторили синтез 113-го элемента,Японские ученые из физического центра RIKEN за...
2,Times: Россия строит новую авиабазу в Сирии,Россия готовится расширить свою военную операц...
3,Власти Египта гарантируют безопасность российс...,Египет ждет российских туристов и гарантирует ...
4,Гордума Новочеркасска приняла отставку мэра го...,В Ростовской области депутаты гордумы Новочерк...


In [10]:
df.shape

(865847, 2)

In [11]:
# Возьму 10к случайных текстов, полный объем считает очень долго
data = df.sample(10000)
data.head()

,title,text
239319,Российский боец поблагодарил фанатов за поддер...,Российский боец смешанного стиля Петр Ян выраз...
147595,Экс-главный тренер «Локомотива» возглавил «Вес...,Экс-главный тренер «Локомотива» Славен Билич в...
691152,В Приамурье из-за пожаров введен режим ЧС,В Амурской области в связи с пожароопасной обс...
218405,«Кузня» проиграла «Автомобилисту» в домашней в...,Новокузнецкий «Металлург» уступил «Автомобилис...
407613,МИД Бельгии заявил о необходимости настоящего ...,Министр иностранных дел БельгииnДидье Рейндерс...


In [12]:
data.shape

(10000, 2)

Препроцессинг текста

In [13]:
assert True

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [14]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [15]:
%%time

with Pool(NUM_THREADS) as p:
    data['processed_text'] = list(tqdm(p.imap(preprocess_txt, data['text']), total=len(data)))

<timed exec>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 19.1 s, sys: 4.21 s, total: 23.3 s
Wall time: 7min 16s


In [16]:
data.head(3)

,title,text,processed_text
239319,Российский боец поблагодарил фанатов за поддер...,Российский боец смешанного стиля Петр Ян выраз...,"[российский, боец, смешанный, стиль, пётр, ян,..."
147595,Экс-главный тренер «Локомотива» возглавил «Вес...,Экс-главный тренер «Локомотива» Славен Билич в...,"[эксглавный, тренер, «локомотива», славный, би..."
691152,В Приамурье из-за пожаров введен режим ЧС,В Амурской области в связи с пожароопасной обс...,"[амурский, область, связь, пожароопасный, обст..."


Создание модели

In [17]:
# Создание модели Word2Vec
%%time
modelW2V = Word2Vec(data['processed_text'], size=300, seed=RANDOM_STATE, window=3, workers=NUM_THREADS, min_count=2)

CPU times: user 31.7 s, sys: 178 ms, total: 31.8 s
Wall time: 20.4 s


In [18]:
# Создание модели FastText
%%time
modelFT = FastText(data['processed_text'], size=300, min_count=2, window=3, seed=RANDOM_STATE, workers=NUM_THREADS)

CPU times: user 2min 16s, sys: 5.9 s, total: 2min 21s
Wall time: 1min 23s


In [19]:
%%time
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

counter = 0
index_map = {}

for line in data['text']:
    n_w2v = 0
    n_ft = 0
    index_map[counter] = line
    text = preprocess_txt(line)
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in text:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

CPU times: user 7min 2s, sys: 627 ms, total: 7min 2s
Wall time: 7min 3s


True

In [20]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

Проверка работы модели

In [21]:
TEXT = "чемпионат мира по футболу"

In [22]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Женская сборная России по биатлону финишировала пятой в эстафете на шестом этапе Кубка мира в итальянском Антхольце, передаетnUra.run.nВ составе команды выступали Виктория Сливко, Татьяна Акимова, Екатерина Глазырина и Ольга Подчуфарова.nЗолото завоевали немецкие спортсменки, серебро — Франция, а бронзу — Италия.nБиатлон. Кубок мира. Шестой этап. Антхольц, ИталияnnЖенщины. Эстафета 4x6 кмn1. Германия — 1:13:57.2.nn2. Франция +24.2.nn3. Италия +33.4.nn«Газета.Ru» велаnтекстовую онлайн-трансляциюnгонки.',
 'Россиянка Валерия Васнецова завоевала золотую медаль чемпионата мира по биатлону среди юниоров в гонке преследования.nВторой стала Микела Каррара из Италии, замкнула тройку призеров Ингрид Тандревольд из Норвегии.nПобеда в пасьюте стала для россиянки первой личной золотой медалью на крупных международных стартах.nБиатлон. Чемпионат мира среди юниоров.nnЮниорки. Гонка преследования. 12,5 км.nn1. Валерия Васнецова (Россия) — 31.49,5.nn2 . Микела Каррара (Италия) — +10,0.nn3. Ингрид Та

In [23]:
get_response(TEXT, ft_index, modelFT, index_map)

['Известный немецкий политологnАлександр Рарnподелился мыслями после вылета сборной Германии с чемпионата мира — 2018, передаетnНСН.nДействующие чемпионы мира потерпели поражение от команды Южной Кореи (0:2) в заключительном матче третьего тура мундиаля, лишившись шансов на выход в плей-офф турнира.n«В Германии футбол – это национальный спорт. Более важного спорта, который смотрят такое количество людей, миллионы людей, в Германии просто нет. Для Германии чемпионат мира по футболу важнее, чем Олимпийские игры. Там проиграть против Южной Кореи, против Мексики и еле-еле выиграть против Швеции и уже в первой групповой фазе вылететь из турнира — это неописуемая катастрофа! Германия в полном трауре!» — заявил Рар.nРанее сообщалось, чтоnэкс-тренер сборной России считает закономерным поражение Германии от Южной Кореи.',
 'Сборная России взяла золото в женской эстафете на четвертом этапе Кубка мира по биатлону, сообщаетn«Советский спорт».nНациональную команду в эстафете представилиnЕвгения Пав